# Task 1: Strategy Development and Backtesting

This notebook implements the following tasks:
1. **Strategy Development**: Create three different trading strategies:
   - RSI Momentum Strategy
   - Moving Average Crossover Strategy
   - Bollinger Bands Crossover Strategy
2. **Backtesting**: Test each strategy using historical OHLC data, and evaluate performance using the following metrics:
   - Sharpe Ratio
   - Sortino Ratio
   - Max Drawdown
   - Annualized Return
   - Cumulative Return
3. **Portfolio Analysis**: Combine all three strategies into a portfolio and analyze the overall performance.


In [1]:
# Import necessary libraries
import backtrader as bt
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [3]:

# Load Data
dataframe = pd.read_csv(r"C:\Users\adity\OneDrive\Desktop\Quant trading project\task_dataset\datasets\quant\portfolio-universe\US\AAPL\AAPL_1d.csv")

# Select only the relevant columns for Backtrader
dataframe = dataframe[['timestamp', 'open', 'high', 'low', 'close', 'volume']]

# Convert 'timestamp' to datetime object for Backtrader
dataframe['timestamp'] = pd.to_datetime(dataframe['timestamp'])

# Set 'timestamp' as the index (required by Backtrader)
dataframe.set_index('timestamp', inplace=True)

# Load the data into Backtrader's PandasData format
data = bt.feeds.PandasData(dataname=dataframe)

# Create a Cerebro entity
cerebro = bt.Cerebro(stdstats=False)


## Strategy 1: RSI Momentum Strategy

The **RSI Momentum Strategy** uses the Relative Strength Index (RSI) to detect overbought and oversold market conditions:
- **Buy**: When RSI crosses below 30 (indicating oversold).
- **Sell**: When RSI crosses above 70 (indicating overbought).


In [5]:
# Define RSI Strategy
class RSIStrategy(bt.Strategy):
    params = dict(period=14, oversold=30, overbought=70)

    def __init__(self):
        self.rsi = bt.indicators.RSI(period=self.p.period)
        self.order = None
        self.trades = []  # Store trade details

    def next(self):
        if self.position.size == 0:
            if self.rsi < self.p.oversold:
                self.order = self.buy(size=100)
                self.trades.append({
                    'entry_date': self.data.datetime.date(0),
                    'entry_price': self.data.close[0],
                    'size': 100
                })
        else:
            if self.rsi > self.p.overbought:
                self.order = self.close()
                self.trades[-1]['exit_date'] = self.data.datetime.date(0)
                self.trades[-1]['exit_price'] = self.data.close[0]
                self.trades[-1]['pnl'] = (self.data.close[0] - self.trades[-1]['entry_price']) * self.trades[-1]['size']


# Moving Average Crossover Strategy

The `MACrossoverStrategy` implements a trading strategy where a short-term moving average (SMA) is compared with a long-term moving average. The strategy:

- Buys an asset when the short-term SMA crosses above the long-term SMA (bullish crossover).
- Sells the asset when the short-term SMA crosses below the long-term SMA (bearish crossover).
- Tracks trades by recording entry and exit dates, prices, and profit/loss (PnL).


In [7]:
# Define Moving Average Crossover Strategy
class MACrossoverStrategy(bt.Strategy):
    params = dict(short_period=50, long_period=200)

    def __init__(self):
        sma_short = bt.indicators.SMA(period=self.p.short_period)
        sma_long = bt.indicators.SMA(period=self.p.long_period)
        self.crossover = bt.indicators.CrossOver(sma_short, sma_long)
        self.order = None
        self.trades = []

    def next(self):
        if self.position.size == 0:
            if self.crossover > 0:
                self.order = self.buy(size=100)
                self.trades.append({
                    'entry_date': self.data.datetime.date(0),
                    'entry_price': self.data.close[0],
                    'size': 100
                })
        else:
            if self.crossover < 0:
                self.order = self.close()
                self.trades[-1]['exit_date'] = self.data.datetime.date(0)
                self.trades[-1]['exit_price'] = self.data.close[0]
                self.trades[-1]['pnl'] = (self.data.close[0] - self.trades[-1]['entry_price']) * self.trades[-1]['size']


# Bollinger Bands Strategy

The `BollingerBandsStrategy` uses Bollinger Bands to make trading decisions. The strategy:

- Buys when the price moves below the lower Bollinger Band (indicating oversold conditions).
- Sells when the price rises above the upper Bollinger Band (indicating overbought conditions).
- Tracks trades similarly by logging entry/exit details and PnL.

In [9]:
# Define Bollinger Bands Strategy
class BollingerBandsStrategy(bt.Strategy):
    params = dict(period=20, devfactor=2)

    def __init__(self):
        self.bb = bt.indicators.BollingerBands(period=self.p.period, devfactor=self.p.devfactor)
        self.order = None
        self.trades = []

    def next(self):
        if self.position.size == 0:
            if self.data.close < self.bb.lines.bot:
                self.order = self.buy(size=100)
                self.trades.append({
                    'entry_date': self.data.datetime.date(0),
                    'entry_price': self.data.close[0],
                    'size': 100
                })
        else:
            if self.data.close > self.bb.lines.top:
                self.order = self.close()
                self.trades[-1]['exit_date'] = self.data.datetime.date(0)
                self.trades[-1]['exit_price'] = self.data.close[0]
                self.trades[-1]['pnl'] = (self.data.close[0] - self.trades[-1]['entry_price']) * self.trades[-1]['size']


# Performance Evaluation

In [11]:

# Performance Analyzer
class PerformanceAnalyzer(bt.Analyzer):
    def __init__(self):
        self.rets = []

    def next(self):
        self.rets.append(self.strategy.broker.get_value())

    def get_analysis(self):
        returns = np.diff(self.rets) / self.rets[:-1]
        cumulative_return = (self.rets[-1] / self.rets[0]) - 1
        annualized_return = (1 + cumulative_return) ** (252 / len(returns)) - 1
        sharpe_ratio = np.mean(returns) / np.std(returns) * np.sqrt(252)
        downside_returns = [r for r in returns if r < 0]
        sortino_ratio = np.mean(returns) / (np.std(downside_returns) * np.sqrt(252))
        max_drawdown = np.min(self.rets / np.maximum.accumulate(self.rets) - 1)
        return {
            'Cumulative Return': cumulative_return,
            'Annualized Return': annualized_return,
            'Sharpe Ratio': sharpe_ratio,
            'Sortino Ratio': sortino_ratio,
            'Max Drawdown': max_drawdown
        }


## Backtesting the Strategies

We will now run the backtest for each of the three strategies using Backtrader.


In [13]:

# Function to run a strategy
def run_strategy(strategy):
    cerebro = bt.Cerebro()
    cerebro.addstrategy(strategy)
    cerebro.adddata(data)
    cerebro.broker.setcash(100000.0)
    cerebro.addanalyzer(PerformanceAnalyzer, _name='perf')
    cerebro.broker.setcommission(commission=0.001)
    result = cerebro.run()
    analyzer = result[0].analyzers.perf.get_analysis()
    return analyzer, cerebro


In [19]:
# Run strategies
strategies = [RSIStrategy, MACrossoverStrategy, BollingerBandsStrategy]
results = {}
trades_summary = []  # Initialize trade summary list
for strat in strategies:
    analysis, cerebro = run_strategy(strat)
    results[strat.__name__] = analysis

    # Collect trade details from strategy
    trades = cerebro.runstrats[0][0].trades
    for trade in trades:
        trades_summary.append({
            'Strategy': strat.__name__,
            'Entry Date': trade['entry_date'],
            'Entry Price': trade['entry_price'],
            'Size': trade['size'],
            'Exit Date': trade.get('exit_date', None),
            'Exit Price': trade.get('exit_price', None),
            'P&L': trade.get('pnl', None)
        })

    print(f"Strategy: {strat.__name__}")
    for key, value in analysis.items():
        print(f"  {key}: {value:.4f}")

    # Plot equity curve
    cerebro.plot(iplot=False)


Strategy: RSIStrategy
  Cumulative Return: 0.0800
  Annualized Return: 0.0052
  Sharpe Ratio: 0.3881
  Sortino Ratio: 0.0011
  Max Drawdown: -0.0287


<IPython.core.display.Javascript object>

Strategy: MACrossoverStrategy
  Cumulative Return: 0.1337
  Annualized Return: 0.0086
  Sharpe Ratio: 0.4588
  Sortino Ratio: 0.0017
  Max Drawdown: -0.0388


<IPython.core.display.Javascript object>

Strategy: BollingerBandsStrategy
  Cumulative Return: 0.0772
  Annualized Return: 0.0051
  Sharpe Ratio: 0.3242
  Sortino Ratio: 0.0010
  Max Drawdown: -0.0341


<IPython.core.display.Javascript object>

## Portfolio Analysis

Now that we have backtested the individual strategies, we will combine the three strategies into a portfolio and analyze its performance.


In [21]:
# Portfolio Analysis
# Combine strategies (for simplicity, equal allocation to each strategy)
portfolio_returns = []
for strat in strategies:
    analysis, _ = run_strategy(strat)
    portfolio_returns.append(analysis['Cumulative Return'])

portfolio_cumulative_return = np.mean(portfolio_returns)
print(f"Portfolio Cumulative Return: {portfolio_cumulative_return:.4f}")

# Correlation Matrix
def get_returns(strategy):
    _, cerebro = run_strategy(strategy)
    rets = cerebro.runstrats[0][0].analyzers.perf.rets
    returns = np.diff(rets) / rets[:-1]
    return returns

returns_data = {strat.__name__: get_returns(strat) for strat in strategies}
returns_df = pd.DataFrame(returns_data)
corr_matrix = returns_df.corr()
print("Correlation Matrix:")
print(corr_matrix)


Portfolio Cumulative Return: 0.0969
Correlation Matrix:
                        RSIStrategy  MACrossoverStrategy  \
RSIStrategy                1.000000             0.365891   
MACrossoverStrategy        0.365891             1.000000   
BollingerBandsStrategy     0.553098             0.535331   

                        BollingerBandsStrategy  
RSIStrategy                           0.553098  
MACrossoverStrategy                   0.535331  
BollingerBandsStrategy                1.000000  


In [23]:
# Convert trades summary to DataFrame and save to CSV
trades_df = pd.DataFrame(trades_summary)
trades_df.to_csv("Trades.csv", index=False)  # Save trades to CSV


# Conclusion

In this notebook, we developed and backtested three trading strategies: RSI Momentum, SMA Crossover, and MACD Crossover. We evaluated the performance of each strategy using key metrics such as Sharpe Ratio, Max Drawdown, Annualized Return, and Cumulative Return. Finally, we combined the strategies into a single portfolio and analyzed its performance.

The final backtest results and performance metrics give insights into which strategies perform better in different market conditions.
